<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/latent_extractor_noUP_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install ENV



In [5]:
!pip install numpy==1.24.4 --force-reinstall

  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.

In [2]:
# 최신 PyTorch 환경 (Colab에서 안정적인 1.10)
!pip install torch==2.1.0 torchvision==0.16.0

# 필수 라이브러리
!pip install dlib opencv-python scikit-image tqdm pillow matplotlib imageio gdown

# 선택 라이브러리 (e4e에서 내부적으로 사용 가능)
!pip install tensorboardX tensorboard ninja pyaml

!pip install face-alignment dlib
!pip install imageio gdown

In [3]:
!pip install mediapipe==0.10.9 numpy==1.24.4 protobuf==3.20.3

In [4]:
!pip install pyrallis

In [5]:
!pip install tqdm requests
!pip install face-alignment
!pip install imageio[ffmpeg]

In [6]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -P /content/pretrained_models/
!bzip2 -d /content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2

--2025-06-16 17:09:22--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-06-16 17:09:22--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  14.2MB/s    in 5.1s    

2025-06-16 17:09:28 (11.9 MB/s) - ‘/content/pretrained_models/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



# prepare project

In [7]:
!git clone https://github.com/omertov/encoder4editing.git

fatal: destination path 'encoder4editing' already exists and is not an empty directory.


In [8]:
!mkdir -p /content/extract_videos
!mkdir -p /content/input_videos
!mkdir -p /content/alignmented_videos
!mkdir -p /content/preprocess_videos
!mkdir -p /content/latent_videos
!mkdir -p /content/alignmented_videos_aligned
!mkdir -p /content/alignmented_videos_cropped
!mkdir -p /content/alignmented_videos_transforms
!mkdir -p /content/alignmented_videos_up
!mkdir -p /content/front_videos

In [9]:
!git clone https://github.com/NVlabs/stylegan3.git

fatal: destination path 'stylegan3' already exists and is not an empty directory.


In [10]:
!mkdir -p /content/pretrained_models

In [11]:
!gdown --id 1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm -O /content/pretrained_models/restyle_e4e_sg3.pt
!gdown --id 13q6m-bpe3Ws9en9y45JEx2PHQirStt8N -O /content/pretrained_models/stylegan3-ffhq-1024x1024.pt
!gdown --id 1KW7bjndL3QG3sxBbZxreGHigcCCpsDgn -O /content/pretrained_models/model_ir_se50.pth

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm
From (redirected): https://drive.google.com/uc?id=1z_cB187QOc6aqVBdLvYvBjoc93-_EuRm&confirm=t&uuid=4d73c821-12fd-4621-b87d-efcacade549f
To: /content/pretrained_models/restyle_e4e_sg3.pt
100% 809M/809M [00:07<00:00, 110MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13q6m-bpe3Ws9en9y45JEx2PHQirStt8N
From (redirected): https://drive.google.com/uc?id=13q6m-bpe3Ws9en9y45JEx2PHQirStt8N&confirm=t&uuid=4f0811ee-7bf9-45ad-8bb3-2bbb6b538da9
To: 

# frame extract

In [12]:
#@title 병렬 프레임 추출 코드 (멀티프로세싱 적용 + 안정 실행)
import cv2
import os
from glob import glob
from tqdm import tqdm
import multiprocessing as mp

video_dir = "/content/input_videos"  #@param {type:"string"}
output_base = "/content/extract_videos"  #@param {type:"string"}
extract_per_sec = 5  #@param {type:"integer"}

os.makedirs(output_base, exist_ok=True)

def extract_video_frames(video_path):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join(output_base, video_name)
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = max(1, int(fps // extract_per_sec))

    frame_idx = 0
    saved_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % interval == 0:
            frame_path = os.path.join(output_dir, f"key_{saved_idx:04d}.png")
            cv2.imwrite(frame_path, frame)
            saved_idx += 1
        frame_idx += 1

    cap.release()
    return f"{video_name}: {saved_idx} frames saved"

if __name__ == "__main__":
    video_paths = glob(os.path.join(video_dir, "*.mp4"))
    print(f"🎬 총 {len(video_paths)}개의 영상 병렬 처리 시작")

    with mp.Pool(processes=mp.cpu_count()) as pool:
        for result in tqdm(pool.imap_unordered(extract_video_frames, video_paths), total=len(video_paths), desc="📦 Processing videos"):
            pass


🎬 총 12개의 영상 병렬 처리 시작


📦 Processing videos: 100%|██████████| 12/12 [00:15<00:00,  1.29s/it]


In [15]:
!pip uninstall tensorflow

!pip install tensorflow

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/lib/python3.11/dist-packages/tensorflow-2.19.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/tensorflow/*
Proceed (Y/n)? ERROR: Operation cancelled by user
  Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


In [1]:
import cv2, os, math, numpy as np, pandas as pd
from glob import glob
from tqdm import tqdm
import mediapipe as mp

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
FACE_MESH = mp_face_mesh.FaceMesh(static_image_mode=True)

# EAR 계산용 눈 좌표 인덱스 (MediaPipe 468 기준)
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
POSE_IDX = [1, 152, 33, 263, 61, 291]

# EAR 기준 보정값 (dlib 기준 0.275 ↔ MediaPipe 0.23 정도)
EAR_THRESHOLD = 0.23
YAW_T = 50
PITCH_T = 55

model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -330.0, -65.0),
    (-225.0, 170.0, -135.0),
    (225.0, 170.0, -135.0),
    (-150.0, -150.0, -125.0),
    (150.0, -150.0, -125.0)
], dtype="double")

def get_mediapipe_landmarks(img):
    h, w = img.shape[:2]
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = FACE_MESH.process(rgb)
    if not res.multi_face_landmarks:
        return None
    lm = res.multi_face_landmarks[0]
    coords = np.array([[p.x * w, p.y * h] for p in lm.landmark])
    return coords

def estimate_pose_mediapipe(landmarks, img_shape):
    image_points = landmarks[POSE_IDX]
    focal = img_shape[1]
    center = (img_shape[1]/2, img_shape[0]/2)
    cam = np.array([[focal, 0, center[0]], [0, focal, center[1]], [0, 0, 1]], dtype="double")
    dist = np.zeros((4,1))
    success, rv, _ = cv2.solvePnP(model_points, image_points, cam, dist)
    return rv if success else None

def rotation_vector_to_euler(rv):
    rmat, _ = cv2.Rodrigues(rv)
    proj = np.hstack((rmat, np.zeros((3,1))))
    angles = cv2.decomposeProjectionMatrix(proj)[6]
    pitch = math.degrees(math.asin(math.sin(math.radians(angles[1][0]))))
    yaw   = math.degrees(math.asin(math.sin(math.radians(angles[2][0]))))
    roll  = -math.degrees(math.asin(math.sin(math.radians(angles[0][0]))))
    return pitch, yaw, roll

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# 경로 설정
INPUT_ROOT = "/content/extract_videos"
OUTPUT_ROOT = "/content/alignmented_videos"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

file_ext = ".png"
video_dirs = [d for d in os.listdir(INPUT_ROOT) if os.path.isdir(os.path.join(INPUT_ROOT, d))]
missing_videos = []

print(f"📦 총 {len(video_dirs)}개 영상 처리 시작")

for video_name in tqdm(video_dirs, desc="🎯 Selecting best frame"):
    input_dir = os.path.join(INPUT_ROOT, video_name)
    output_dir = os.path.join(OUTPUT_ROOT, video_name)

    candidates = []

    for f in sorted(os.listdir(input_dir)):
        if not f.lower().endswith(file_ext):
            continue

        full_path = os.path.join(input_dir, f)
        img = cv2.imread(full_path)
        if img is None:
            continue

        lm = get_mediapipe_landmarks(img)
        if lm is None:
            continue

        rv = estimate_pose_mediapipe(lm, img.shape)
        if rv is None:
            continue

        pitch, yaw, _ = rotation_vector_to_euler(rv)

        left_eye = lm[LEFT_EYE_IDX]
        right_eye = lm[RIGHT_EYE_IDX]
        avg_EAR = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0

        candidates.append({
        'img': img,
        'filename': f,
        'ear': avg_EAR,
        'pitch': pitch,
        'yaw': yaw
        })

    best_img = None
    best_filename = None

    if candidates:
        top_ear = sorted(candidates, key=lambda x: -x['ear'])[:13]
        good = [c for c in top_ear if abs(c['pitch']) < 40 and abs(c['yaw']) < 25]

        if good:
            best = min(good, key=lambda x: abs(x['pitch']) + abs(x['yaw']))
        else:
            best = min(candidates, key=lambda x: abs(x['pitch']) + abs(x['yaw']))

        best_img = best['img']
        best_filename = best['filename']

    if best_img is not None:
        save_path = os.path.join(OUTPUT_ROOT, f"{video_name}.png")
        cv2.imwrite(save_path, best_img)
    else:
        missing_videos.append(video_name)

if missing_videos:
    df_missing = pd.DataFrame(missing_videos, columns=["video_name"])
    df_missing.to_csv("no_frame_found.csv", index=False)
    print(f"❗ Missing video list saved to no_frame_found.csv")
else:
    print("✅ All videos had suitable frames.")


📦 총 12개 영상 처리 시작


🎯 Selecting best frame: 100%|██████████| 12/12 [00:20<00:00,  1.69s/it]

✅ All videos had suitable frames.


In [2]:
!rm -rf /content/extract_videos/*

#styleGAN-edting 준비

In [3]:
%cd /content
!git clone https://github.com/yuval-alaluf/stylegan3-editing.git
%cd /content/stylegan3-editing

/content
fatal: destination path 'stylegan3-editing' already exists and is not an empty directory.
/content/stylegan3-editing


In [4]:
!mkdir -p /content/stylegan3-editing/pretrained_models
!cp /content/pretrained_models/shape_predictor_68_face_landmarks.dat /content/stylegan3-editing/pretrained_models/

In [5]:
import sys
sys.path.insert(0, '/content/stylegan3-editing')
sys.path.append('/content/stylegan3-editing')


In [6]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2
!mkdir -p /content/stylegan3-editing/pretrained_models
!mv shape_predictor_68_face_landmarks.dat /content/stylegan3-editing/pretrained_models/

--2025-06-16 17:11:53--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-06-16 17:11:53--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  14.4MB/s    in 4.9s    

2025-06-16 17:11:59 (12.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



# image 정렬

In [7]:
import sys
sys.path.insert(0, '/content/stylegan3-editing')
sys.path.append('/content/stylegan3-editing')
import time
base_raw_root = "/content/alignmented_videos"
aligned_root = f"{base_raw_root}_aligned"
cropped_root = f"{base_raw_root}_croped"
transform_root = f"{base_raw_root}_transforms"

print("🚀 Aligning all images...")
!python /content/stylegan3-editing/prepare_data/preparing_faces_parallel.py \
    --mode align \
    --root_path "{base_raw_root}"

time.sleep(3)

print("🔁 Cropping all images...")
!python /content/stylegan3-editing/prepare_data/preparing_faces_parallel.py \
    --mode crop \
    --root_path "{base_raw_root}" \
    --random_shift 0.05

time.sleep(3)

print("🔁 Computing transforms for all images...")
import os
os.environ['PYTHONPATH'] = '/content/stylegan3-editing'
!python /content/stylegan3-editing/prepare_data/compute_landmarks_transforms.py \
  --raw_root "/content/alignmented_videos" \
  --aligned_root "/content/alignmented_videos_aligned" \
  --cropped_root "/content/alignmented_videos_croped" \
  --output_root "/content/alignmented_videos_transforms"

print("✅ All preprocessing completed!")

🚀 Aligning all images...

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/content/stylegan3-editing/prepare_data/preparing_faces_parallel.py", line 17, in <module>
    from utils.alignment_utils import align_face, crop_face
  File "/content/stylegan3-editing/utils/alignment_utils.py", line 8, in <module>
    from utils.common import make_transform
  File "/content/stylegan3-editing/utils/common.py", line 5, in <module>
    import torch
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 1382, in <module>
    from .functional import *  # noqa

# latent extract

In [2]:
%cd /content/stylegan3-editing/

/content/stylegan3-editing


In [8]:
!pip uninstall numpy -y
!pip install numpy==1.24.4


Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy

In [10]:
import os

import sys
sys.path.insert(0, '/content/stylegan3-editing')
sys.path.append('/content/stylegan3-editing')


# 디렉토리 설정
input_root = "/content/alignmented_videos_croped"
transforms_root = "/content/alignmented_videos_transforms/landmarks_transforms.npy"
output_root = "/content/experiments/restyle_e4e_sg3"
ckpt_path = "/content/pretrained_models/restyle_e4e_sg3.pt"
script_path = "/content/stylegan3-editing/inversion/scripts/inference_iterative.py"

# output 디렉토리 생성
os.makedirs(output_root, exist_ok=True)

print("🚀 Inverting video")

!PYTHONPATH=/content/stylegan3-editing \
python /content/stylegan3-editing/inversion/scripts/inference_iterative.py \
    --output_path "/content/experiments/restyle_e4e_sg3" \
    --checkpoint_path "/content/pretrained_models/restyle_e4e_sg3.pt" \
    --data_path "/content/alignmented_videos_croped" \
    --test_batch_size 4 \
    --test_workers 4 \
    --n_iters_per_batch 3 \
    --landmarks_transforms_path "/content/alignmented_videos_transforms/landmarks_transforms.npy"


🚀 Inverting video
Loading ReStyle e4e from checkpoint: /content/pretrained_models/restyle_e4e_sg3.pt
Loading StyleGAN3 generator from path: None
Done!
Model successfully loaded!
Loading dataset for ffhq_encode
Traceback (most recent call last):
  File "/content/stylegan3-editing/inversion/scripts/inference_iterative.py", line 105, in <module>
    run()
  File "/usr/local/lib/python3.11/dist-packages/pyrallis/argparsing.py", line 158, in wrapper_inner
    response = fn(cfg, *args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan3-editing/inversion/scripts/inference_iterative.py", line 35, in run
    dataset = InferenceDataset(root=opts.data_path,
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan3-editing/inversion/datasets/inference_dataset.py", line 14, in __init__
    self.landmarks_transforms = self._get_landmarks_transforms(landmarks_transforms_path)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# 유사도 비교

In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 1. Load latent dictionary
latent_path = "/content/experiments/restyle_e4e_sg3/latents.npy"
latent_dict = np.load(latent_path, allow_pickle=True).item()

# 2. 정렬된 파일 리스트 확보
filenames = sorted(latent_dict.keys())

# 3. 각 latent에서 마지막 step → 평균 → (512,)
latents = []
for key in filenames:
    latent = latent_dict[key][-1]  # 마지막 step (18, 512)
    mean_latent = latent.mean(axis=0).astype('float32')  # (512,)
    latents.append(mean_latent)

latents = np.stack(latents)  # shape: (N, 512)

# 4. cosine similarity matrix
sim_matrix = cosine_similarity(latents)  # shape: (N, N)

# 5. 각 query 파일마다 top-3 유사한 match + score 저장
rows = []

for i in range(len(filenames)):
    sims = sim_matrix[i].copy()
    sims[i] = -np.inf  # 자기 자신 제외
    top3_idx = np.argsort(sims)[::-1][:3]
    row = {
        "query": filenames[i],
        "top1": filenames[top3_idx[0]],
        "top2": filenames[top3_idx[1]],
        "top3": filenames[top3_idx[2]],
        "top1val": round(float(sims[top3_idx[0]]), 6),
        "top2val": round(float(sims[top3_idx[1]]), 6),
        "top3val": round(float(sims[top3_idx[2]]), 6),
    }
    rows.append(row)

# 6. Save to CSV
df = pd.DataFrame(rows)
df.to_csv("video_similarity_top3_compact.csv", index=False)

print("✅ Saved to video_similarity_top3_compact.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/experiments/restyle_e4e_sg3/latents.npy'

#image generate

In [ ]:
!mkdir -p editing/interfacegan/boundaries/ffhq

In [ ]:
!pip install -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# 🔐 인증
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 📁 드라이브 폴더 ID (입력한 링크 기준)
folder_id = '1tzlY5MY2P6dr9dCnBc7a-9RUs6zUxh3E'
target_dir = '/content/stylegan3-editing/editing/interfacegan/boundaries/ffhq'
os.makedirs(target_dir, exist_ok=True)

# ⬇️ 다운로드
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
for f in file_list:
    print(f"🔽 Downloading {f['title']}")
    f.GetContentFile(os.path.join(target_dir, f['title']))


🔽 Downloading No_Beard_boundary.npy
🔽 Downloading 5_o_Clock_Shadow_boundary.npy
🔽 Downloading Receding_Hairline_boundary.npy
🔽 Downloading Gray_Hair_boundary.npy
🔽 Downloading Bangs_boundary.npy
🔽 Downloading Brown_Hair_boundary.npy
🔽 Downloading Smiling_boundary.npy
🔽 Downloading Sideburns_boundary.npy
🔽 Downloading High_Cheekbones_boundary.npy
🔽 Downloading Wearing_Necklace_boundary.npy
🔽 Downloading Goatee_boundary.npy
🔽 Downloading Straight_Hair_boundary.npy
🔽 Downloading Bags_Under_Eyes_boundary.npy
🔽 Downloading Young_boundary.npy
🔽 Downloading Bushy_Eyebrows_boundary.npy
🔽 Downloading Pointy_Nose_boundary.npy
🔽 Downloading Chubby_boundary.npy
🔽 Downloading Wearing_Earrings_boundary.npy
🔽 Downloading Black_Hair_boundary.npy
🔽 Downloading Wearing_Necktie_boundary.npy
🔽 Downloading Attractive_boundary.npy
🔽 Downloading age_boundary.npy
🔽 Downloading Wavy_Hair_boundary.npy
🔽 Downloading Heavy_Makeup_boundary.npy
🔽 Downloading Male_boundary.npy
🔽 Downloading Big_Nose_boundary.npy
🔽 D

In [ ]:
!python /content/stylegan3-editing/inversion/scripts/inference_editing.py \
--output_path /content/experiments/restyle_e4e_sg3/003_000 \
--checkpoint_path /content/pretrained_models/restyle_e4e_sg3.pt \
--data_path /content/alignmented_videos_cropped/003_000 \
--test_batch_size 4 \
--test_workers 2 \
--n_iters_per_batch 3 \
--edit_directions "[]" \
--factor_ranges "[]" \
--landmarks_transforms_path /content/alignmented_videos_transforms/003_000/landmarks_transforms.npy


Loading ReStyle e4e from checkpoint: /content/pretrained_models/restyle_e4e_sg3.pt
Loading StyleGAN3 generator from path: None
Done!
Model successfully loaded!
Loading dataset for ffhq_encode
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
100% 4/4 [00:22<00:00,  5.67s/it]
Runtime 5.5591+-0.6507


In [ ]:
!zip -r /content/outputs_all.zip /content/experiments/restyle_e4e_sg3


  adding: content/experiments/restyle_e4e_sg3/ (stored 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/ (stored 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0125.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0445.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0287.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0397.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0536.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0322.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0279.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0057.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0020.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/inference_coupled/0440.png (deflate

In [ ]:
from google.colab import files
files.download('/content/restyle_outputs.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/033_000pose.zip /content/experiments/restyle_e4e_sg3/003_000/editing_results/pose

  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/ (stored 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0026.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0003.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0000.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0023.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0028.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0042.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0027.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0008.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/pose/key_0029.png (deflate

In [ ]:
!zip -r /content/033_000age.zip /content/experiments/restyle_e4e_sg3/003_000/editing_results/age

  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/ (stored 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0026.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0003.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0000.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0023.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0028.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0042.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0027.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0008.png (deflated 0%)
  adding: content/experiments/restyle_e4e_sg3/003_000/editing_results/age/key_0029.png (deflated 0%)
  ad